## Datathon-2: Notebook Submission
This is a template for Assignment 3 of Machine Learning Bootcamp. Please make sure you perform all tasks within the sections created. This ensures that your assignment is evaluated correctly. 


## Task 1

### Import Libraries

In [ ]:
!pip install boruta
!pip install pycaret
import pandas as pd
import numpy as np
import sklearn as sk
from pycaret.regression import *
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV 

### Load the data and display first 5 rows.

In [2]:
cancer_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv" )
cancer_data.head(5)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,Geography,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctSomeCol18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,TARGET_deathRate
0,19.0,8,481.5,50038,2704,11.1,0.0,"(48021.6, 51046.4]",48.4,49.6,46.4,"Hettinger County, North Dakota",2.25,65.9,10.8,25.0,57.4,6.8,37.1,12.2,57.4,1.1,81.2,56.0,35.7,34.7,9.9,96.032049,0.724914,0.000000,0.000000,62.511457,15.157116,160.3
1,88.0,34,486.0,59399,14844,9.7,0.0,"(54545.6, 61494.5]",41.9,41.3,43.2,"Mills County, Iowa",2.63,58.6,22.3,29.1,NaN,1.1,35.9,16.0,60.4,3.8,76.7,NaN,50.8,32.1,12.8,97.537344,0.719957,0.080743,0.040371,61.641045,3.293510,194.9
2,195.0,83,475.7,39721,25164,18.5,0.0,"(37413.8, 40362.7]",48.9,47.9,49.9,"Gladwin County, Michigan",2.30,57.2,24.9,36.2,NaN,3.5,40.2,7.6,41.2,11.0,61.6,NaN,32.1,49.8,21.6,97.576566,0.360770,0.411749,0.082350,53.978102,6.390328,196.5
3,116.0,55,496.6,30299,17917,28.1,0.0,"[22640, 34218.1]",44.2,42.7,45.2,"Fentress County, Tennessee",2.43,53.0,10.9,51.8,NaN,5.3,44.2,7.0,41.6,10.4,45.2,NaN,24.2,53.2,33.0,97.908650,0.161731,0.306731,0.340193,51.013143,5.124836,230.9
4,80.0,35,372.0,39625,14058,17.4,0.0,"(37413.8, 40362.7]",45.0,42.2,48.0,"Las Animas County, Colorado",2.36,52.2,12.6,31.2,NaN,0.2,28.3,10.5,49.3,9.2,56.6,NaN,33.4,43.0,24.7,82.672551,1.834103,0.682617,8.253465,50.566426,3.897033,162.2


### Perform Exploratory Data Analysis

In [4]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              3051 non-null   float64
 1   avgDeathsPerYear         3051 non-null   int64  
 2   incidenceRate            3051 non-null   float64
 3   medIncome                3051 non-null   int64  
 4   popEst2015               3051 non-null   int64  
 5   povertyPercent           3051 non-null   float64
 6   studyPerCap              3051 non-null   float64
 7   binnedInc                3051 non-null   object 
 8   MedianAge                3051 non-null   float64
 9   MedianAgeMale            3051 non-null   float64
 10  MedianAgeFemale          3051 non-null   float64
 11  Geography                3051 non-null   object 
 12  AvgHouseholdSize         3051 non-null   float64
 13  PercentMarried           3051 non-null   float64
 14  PctNoHS18_24            

**MissingValues**
---
PctSomeCol18_24          785 non-null    float64


---


PctPrivateCoverageAlone  2447 non-null   float64

---


PctEmployed16_Over       2899 non-null   float64



In [5]:
cancer_data.describe()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctSomeCol18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,TARGET_deathRate
count,3051.000000,3051.000000,3051.000000,3051.000000,3.051000e+03,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,785.000000,3051.000000,3051.000000,3051.000000,2899.000000,3051.000000,3051.000000,2447.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000,3051.000000
mean,570.668154,176.000983,449.007042,46902.917076,9.495799e+04,16.880367,158.695936,44.303540,39.600885,42.234579,2.473048,51.885480,18.225139,34.920190,41.247898,6.131957,34.909440,13.212750,54.115626,7.841069,64.519338,48.573314,41.342347,36.287545,19.198820,84.005243,9.166570,1.198561,1.864829,51.355837,5.608851,178.909767
std,1250.546532,445.042777,52.886386,11902.460659,2.761007e+05,6.340462,544.035590,38.704107,5.177629,5.253474,0.429926,6.812846,8.105799,9.021475,11.107006,4.552222,7.015728,5.360342,8.267656,3.454863,10.511932,10.011218,9.327793,7.748442,6.023164,16.126982,14.676772,2.369931,3.235204,6.524964,1.955201,27.570075
min,6.000000,3.000000,211.100000,22640.000000,8.270000e+02,3.200000,0.000000,22.300000,22.400000,22.300000,0.022100,25.100000,0.000000,0.000000,7.100000,0.000000,7.500000,3.200000,17.600000,0.400000,23.400000,16.800000,14.300000,11.800000,2.600000,11.008762,0.000000,0.000000,0.000000,23.915652,0.000000,66.300000
25%,80.000000,29.000000,421.800000,38752.000000,1.236850e+04,12.200000,0.000000,37.900000,36.400000,39.200000,2.370000,47.800000,12.800000,29.300000,34.000000,3.100000,30.650000,9.300000,48.600000,5.500000,57.500000,41.300000,34.700000,31.000000,14.900000,78.012571,0.616576,0.261748,0.282825,47.736828,4.499936,161.400000
50%,171.000000,62.000000,453.549422,45098.000000,2.677700e+04,15.900000,0.000000,41.000000,39.500000,42.400000,2.500000,52.500000,17.200000,34.700000,41.000000,5.300000,35.400000,12.300000,54.400000,7.600000,65.300000,48.700000,41.300000,36.300000,18.800000,90.318790,2.276756,0.557031,0.791571,51.757925,5.384471,178.300000
75%,508.000000,148.000000,481.300000,52410.500000,6.853600e+04,20.400000,86.581336,43.900000,42.500000,45.300000,2.630000,56.500000,22.600000,40.700000,46.900000,8.100000,39.700000,16.100000,60.300000,9.700000,72.200000,55.700000,47.700000,41.400000,23.000000,95.577396,10.326954,1.189955,2.080241,55.465803,6.473896,195.500000
max,24965.000000,9445.000000,1206.900000,125635.000000,5.238216e+06,47.000000,9762.308998,525.600000,64.700000,65.700000,3.930000,72.500000,64.100000,72.500000,79.000000,51.800000,54.800000,40.400000,76.500000,29.400000,92.300000,78.900000,70.700000,65.100000,46.600000,100.000000,84.866024,35.640183,38.743747,71.703057,21.326165,362.800000


In [10]:
new_data = cancer_data.drop_duplicates() #removing duplicate values
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285 entries, 0 to 2284
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              2285 non-null   float64
 1   avgDeathsPerYear         2285 non-null   int64  
 2   incidenceRate            2285 non-null   float64
 3   medIncome                2285 non-null   int64  
 4   popEst2015               2285 non-null   int64  
 5   povertyPercent           2285 non-null   float64
 6   studyPerCap              2285 non-null   float64
 7   binnedInc                2285 non-null   object 
 8   MedianAge                2285 non-null   float64
 9   MedianAgeMale            2285 non-null   float64
 10  MedianAgeFemale          2285 non-null   float64
 11  Geography                2285 non-null   object 
 12  AvgHouseholdSize         2285 non-null   float64
 13  PercentMarried           2285 non-null   float64
 14  PctNoHS18_24            

In [15]:
new_data.isna().sum()

avgAnnCount                   0
avgDeathsPerYear              0
incidenceRate                 0
medIncome                     0
popEst2015                    0
povertyPercent                0
studyPerCap                   0
binnedInc                     0
MedianAge                     0
MedianAgeMale                 0
MedianAgeFemale               0
Geography                     0
AvgHouseholdSize              0
PercentMarried                0
PctNoHS18_24                  0
PctHS18_24                    0
PctSomeCol18_24            1705
PctBachDeg18_24               0
PctHS25_Over                  0
PctBachDeg25_Over             0
PctEmployed16_Over          119
PctUnemployed16_Over          0
PctPrivateCoverage            0
PctPrivateCoverageAlone     461
PctEmpPrivCoverage            0
PctPublicCoverage             0
PctPublicCoverageAlone        0
PctWhite                      0
PctBlack                      0
PctAsian                      0
PctOtherRace                  0
PctMarri

newdata missing values

---


PctSomeCol18_24            1705

---

PctEmployed16_Over          119

---

PctPrivateCoverageAlone     461


In [17]:
clean_data = new_data.drop(columns="PctSomeCol18_24")#eighty percent of data is missing
clean_data.columns

Index(['avgAnnCount', 'avgDeathsPerYear', 'incidenceRate', 'medIncome',
       'popEst2015', 'povertyPercent', 'studyPerCap', 'binnedInc', 'MedianAge',
       'MedianAgeMale', 'MedianAgeFemale', 'Geography', 'AvgHouseholdSize',
       'PercentMarried', 'PctNoHS18_24', 'PctHS18_24', 'PctBachDeg18_24',
       'PctHS25_Over', 'PctBachDeg25_Over', 'PctEmployed16_Over',
       'PctUnemployed16_Over', 'PctPrivateCoverage', 'PctPrivateCoverageAlone',
       'PctEmpPrivCoverage', 'PctPublicCoverage', 'PctPublicCoverageAlone',
       'PctWhite', 'PctBlack', 'PctAsian', 'PctOtherRace',
       'PctMarriedHouseholds', 'BirthRate', 'TARGET_deathRate'],
      dtype='object')

In [28]:
clean_data = new_data.drop(columns=["Geography","PctSomeCol18_24"])#nothing to generalize the data 
clean_data.dtypes

avgAnnCount                float64
avgDeathsPerYear             int64
incidenceRate              float64
medIncome                    int64
popEst2015                   int64
povertyPercent             float64
studyPerCap                float64
binnedInc                   object
MedianAge                  float64
MedianAgeMale              float64
MedianAgeFemale            float64
AvgHouseholdSize           float64
PercentMarried             float64
PctNoHS18_24               float64
PctHS18_24                 float64
PctBachDeg18_24            float64
PctHS25_Over               float64
PctBachDeg25_Over          float64
PctEmployed16_Over         float64
PctUnemployed16_Over       float64
PctPrivateCoverage         float64
PctPrivateCoverageAlone    float64
PctEmpPrivCoverage         float64
PctPublicCoverage          float64
PctPublicCoverageAlone     float64
PctWhite                   float64
PctBlack                   float64
PctAsian                   float64
PctOtherRace        

In [29]:
cat_variables = clean_data[[ "binnedInc" ]]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
cat_dummies.head()
cat_dummies.count()

binnedInc_(37413.8, 40362.7]    2285
binnedInc_(40362.7, 42724.4]    2285
binnedInc_(42724.4, 45201]      2285
binnedInc_(45201, 48021.6]      2285
binnedInc_(48021.6, 51046.4]    2285
binnedInc_(51046.4, 54545.6]    2285
binnedInc_(54545.6, 61494.5]    2285
binnedInc_(61494.5, 125635]     2285
binnedInc_[22640, 34218.1]      2285
dtype: int64

In [31]:
df = clean_data.drop(['binnedInc'], axis=1)
df = pd.concat([df, cat_dummies], axis=1)#replacing the new encoded data
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2285 entries, 0 to 2284
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   avgAnnCount                   2285 non-null   float64
 1   avgDeathsPerYear              2285 non-null   int64  
 2   incidenceRate                 2285 non-null   float64
 3   medIncome                     2285 non-null   int64  
 4   popEst2015                    2285 non-null   int64  
 5   povertyPercent                2285 non-null   float64
 6   studyPerCap                   2285 non-null   float64
 7   MedianAge                     2285 non-null   float64
 8   MedianAgeMale                 2285 non-null   float64
 9   MedianAgeFemale               2285 non-null   float64
 10  AvgHouseholdSize              2285 non-null   float64
 11  PercentMarried                2285 non-null   float64
 12  PctNoHS18_24                  2285 non-null   float64
 13  Pct

### Separate the Input and Target Features of the data

In [32]:
y = df['TARGET_deathRate']
x= df.loc[:, df.columns != 'TARGET_deathRate']

### Split the data into Train and Test Sets

In [76]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.3, random_state = 123)


## Task 2

### Perform Data Preparation Steps

In [77]:
# xTrain.info()
xTrain.isnull().sum()

avgAnnCount                       0
avgDeathsPerYear                  0
incidenceRate                     0
medIncome                         0
popEst2015                        0
povertyPercent                    0
studyPerCap                       0
MedianAge                         0
MedianAgeMale                     0
MedianAgeFemale                   0
AvgHouseholdSize                  0
PercentMarried                    0
PctNoHS18_24                      0
PctHS18_24                        0
PctBachDeg18_24                   0
PctHS25_Over                      0
PctBachDeg25_Over                 0
PctEmployed16_Over               84
PctUnemployed16_Over              0
PctPrivateCoverage                0
PctPrivateCoverageAlone         340
PctEmpPrivCoverage                0
PctPublicCoverage                 0
PctPublicCoverageAlone            0
PctWhite                          0
PctBlack                          0
PctAsian                          0
PctOtherRace                

In [78]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_transformed_df = pd.DataFrame(scaler.fit_transform(xTrain), columns = xTrain.columns)
new_transformed_df.head()
new_transformed_df.describe()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,"binnedInc_(37413.8, 40362.7]","binnedInc_(40362.7, 42724.4]","binnedInc_(42724.4, 45201]","binnedInc_(45201, 48021.6]","binnedInc_(48021.6, 51046.4]","binnedInc_(51046.4, 54545.6]","binnedInc_(54545.6, 61494.5]","binnedInc_(61494.5, 125635]","binnedInc_[22640, 34218.1]"
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1515.000000,1599.000000,1599.000000,1259.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,0.022578,0.017858,0.239719,0.236885,0.017423,0.312937,0.016567,0.046149,0.476382,0.535455,0.640470,0.607445,0.291643,0.482573,0.118819,0.579170,0.272486,0.620371,0.280327,0.596920,0.517996,0.480061,0.462142,0.375115,0.821657,0.107487,0.034657,0.046715,0.576681,0.262699,0.100063,0.095059,0.095059,0.095685,0.098812,0.108818,0.107567,0.093809,0.101939
std,0.049463,0.046082,0.056094,0.117095,0.051461,0.147995,0.059735,0.089326,0.142721,0.139829,0.115082,0.153077,0.130608,0.125470,0.091863,0.145042,0.147270,0.142295,0.129848,0.153476,0.164443,0.166507,0.158093,0.137855,0.178747,0.171978,0.068803,0.081532,0.134705,0.092623,0.300177,0.293389,0.293389,0.294251,0.298502,0.311508,0.309930,0.291654,0.302662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.002845,0.002754,0.211990,0.154338,0.002130,0.201373,0.000000,0.031194,0.390582,0.455764,0.612342,0.513636,0.202552,0.404138,0.057915,0.490486,0.162534,0.526316,0.189394,0.492743,0.395765,0.359929,0.350610,0.273349,0.752347,0.007641,0.007561,0.007206,0.502799,0.211093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.006691,0.006143,0.243472,0.220088,0.004994,0.288330,0.000000,0.037155,0.473684,0.538874,0.649013,0.620455,0.275917,0.475862,0.102317,0.587738,0.247934,0.626486,0.272727,0.608128,0.519544,0.473404,0.463415,0.366743,0.893446,0.025896,0.015766,0.020612,0.585309,0.252732,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.020393,0.015251,0.272494,0.291669,0.012852,0.397025,0.008332,0.043314,0.556787,0.621984,0.683064,0.711364,0.366826,0.562759,0.156371,0.676533,0.353994,0.729202,0.352273,0.712627,0.635179,0.595745,0.570122,0.462415,0.950036,0.120380,0.034438,0.051858,0.663038,0.301313,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:
imputer = KNNImputer(n_neighbors=5)
xTrain = pd.DataFrame(imputer.fit_transform(new_transformed_df),columns = new_transformed_df.columns)
xTrain.head()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,"binnedInc_(37413.8, 40362.7]","binnedInc_(40362.7, 42724.4]","binnedInc_(42724.4, 45201]","binnedInc_(45201, 48021.6]","binnedInc_(48021.6, 51046.4]","binnedInc_(51046.4, 54545.6]","binnedInc_(54545.6, 61494.5]","binnedInc_(61494.5, 125635]","binnedInc_[22640, 34218.1]"
0,0.008374,0.007308,0.297449,0.382048,0.009851,0.176201,0.000000,0.026823,0.357341,0.378016,0.688302,0.693182,0.239234,0.486897,0.142857,0.490486,0.380165,0.809847,0.246212,0.754717,0.768730,0.776596,0.229675,0.255125,0.871837,0.061103,0.024660,0.068927,0.692743,0.246559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.064826,0.061534,0.295642,0.306500,0.052200,0.272311,0.004483,0.036360,0.457064,0.528150,0.685683,0.454545,0.196172,0.446897,0.181467,0.536998,0.374656,0.685908,0.458333,0.631350,0.539088,0.549645,0.404472,0.341686,0.630805,0.183393,0.232504,0.131811,0.439929,0.220228,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.013222,0.015781,0.200944,0.208486,0.009571,0.329519,0.010051,0.050864,0.645429,0.734584,0.617581,0.581818,0.312600,0.553103,0.027027,0.429175,0.413223,0.539898,0.287879,0.506531,0.338762,0.262411,0.589431,0.375854,0.914942,0.019267,0.023946,0.067320,0.515427,0.247181,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.016227,0.014827,0.246435,0.277052,0.017343,0.237986,0.000000,0.033181,0.418283,0.485255,0.685683,0.670455,0.385965,0.357241,0.110039,0.520085,0.325069,0.607810,0.280303,0.669086,0.630293,0.620567,0.333333,0.298405,0.787103,0.151608,0.034944,0.052723,0.711293,0.243058,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.006010,0.006460,0.214501,0.209088,0.009898,0.372998,0.000000,0.018081,0.229917,0.265416,0.761643,0.581818,0.333333,0.404138,0.100386,0.422833,0.314050,0.478778,0.481061,0.541364,0.472313,0.164894,0.439024,0.451025,0.398622,0.402373,0.036917,0.113299,0.587997,0.345441,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
xTrain.isnull().sum()

In [ ]:
xTest.isnull().sum()

In [82]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_transformed_df1 = pd.DataFrame(scaler.fit_transform(xTest), columns = xTest.columns)
new_transformed_df1.head()
new_transformed_df1.describe()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,"binnedInc_(37413.8, 40362.7]","binnedInc_(40362.7, 42724.4]","binnedInc_(42724.4, 45201]","binnedInc_(45201, 48021.6]","binnedInc_(48021.6, 51046.4]","binnedInc_(51046.4, 54545.6]","binnedInc_(54545.6, 61494.5]","binnedInc_(61494.5, 125635]","binnedInc_[22640, 34218.1]"
count,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,651.000000,686.000000,686.000000,565.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000,686.000000
mean,0.049465,0.043915,0.529329,0.232728,0.036382,0.301877,0.034571,0.038823,0.382708,0.453588,0.630243,0.545444,0.277724,0.481654,0.204209,0.563352,0.267183,0.598933,0.246335,0.589465,0.461918,0.487668,0.454756,0.423792,0.810931,0.115426,0.035522,0.055119,0.571970,0.343958,0.110787,0.106414,0.118076,0.103499,0.096210,0.088921,0.099125,0.099125,0.093294
std,0.094040,0.095654,0.111496,0.122683,0.087825,0.141589,0.103079,0.058988,0.125304,0.120888,0.110644,0.153428,0.126520,0.163037,0.144515,0.186415,0.144921,0.147039,0.119515,0.175500,0.173462,0.168698,0.141413,0.150733,0.191382,0.180059,0.075046,0.095234,0.142088,0.128278,0.314097,0.308592,0.322933,0.304831,0.295094,0.284838,0.299048,0.299048,0.291057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.006442,0.006421,0.471077,0.156995,0.004262,0.198614,0.000000,0.028800,0.303178,0.383065,0.600809,0.454746,0.195362,0.378815,0.100671,0.449239,0.163978,0.498141,0.167832,0.477966,0.338104,0.370849,0.362101,0.319588,0.735536,0.007642,0.007074,0.008438,0.496215,0.268241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.014842,0.014695,0.546963,0.210631,0.009795,0.284065,0.000000,0.035300,0.378973,0.458525,0.634075,0.561810,0.259434,0.482944,0.177852,0.576142,0.239247,0.600372,0.237762,0.598305,0.466905,0.497232,0.457786,0.414948,0.879791,0.032440,0.015544,0.022979,0.579430,0.328299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.043175,0.037417,0.600802,0.282577,0.025122,0.375866,0.022628,0.040800,0.452323,0.523041,0.667341,0.648455,0.333333,0.583483,0.271812,0.690355,0.336022,0.702602,0.307692,0.710169,0.579606,0.593635,0.549250,0.512887,0.952085,0.138618,0.032337,0.058689,0.661551,0.410137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
imputer = KNNImputer(n_neighbors=5)
xTest = pd.DataFrame(imputer.fit_transform(new_transformed_df1),columns = new_transformed_df1.columns)
xTest.head()
xTest.info()

## Task 3

### Build any Machine Learning Model of your choice

In [85]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(xTrain, yTrain)
reg

LinearRegression()

In [87]:
y_pred = reg.predict(xTest)  


### Evaluate your model with various Evaluation Metrics

In [88]:
#xTrain, xTest, yTrain, yTest
from sklearn import metrics
# print result of MAE
print(metrics.mean_absolute_error(yTest, y_pred))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred)))

57.04637097627063
3763.4739432374395
61.3471592108179


### Try out other Machine Learning Models and Evaluate them

In [89]:
regressor1 = RandomForestRegressor(n_estimators = 100, random_state = 1,max_depth=5)
regressor2 = RandomForestRegressor(n_estimators = 100, random_state = 1,max_depth=10)
regressor3 = RandomForestRegressor(n_estimators = 100, random_state = 1,max_depth=15)

In [90]:
regressor1.fit(xTrain, yTrain)
regressor2.fit(xTrain, yTrain)
regressor3.fit(xTrain, yTrain)

RandomForestRegressor(max_depth=15, random_state=1)

In [91]:
y_pred_regressor1 = regressor1.predict(xTest)  

In [92]:
#xTrain, xTest, yTrain, yTest
from sklearn import metrics
# print result of MAE
print(metrics.mean_absolute_error(yTest, y_pred_regressor1))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred_regressor1))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred_regressor1)))

23.995912986394096
1001.0531660271007
31.639424236656087


In [93]:
y_pred_regressor2 = regressor2.predict(xTest)  

In [94]:
#xTrain, xTest, yTrain, yTest
from sklearn import metrics
# print result of MAE
print(metrics.mean_absolute_error(yTest, y_pred_regressor2))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred_regressor2))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred_regressor2)))

25.34731263766748
1113.2772492676784
33.36580958507793


In [95]:
y_pred_regressor3 = regressor3.predict(xTest)  

In [97]:
print(metrics.mean_absolute_error(yTest, y_pred_regressor3))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred_regressor3))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred_regressor3)))

25.555439638326586
1129.512754166359
33.60822450184417


## Task 4

### Perform Hyperparameter Tuning

In [125]:
# defining parameter range 
param_grid = { 'max_depth': [1, 2,3,4,5,6,7],

'n_estimators': [100, 200] 
}  
  
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 2, cv = 3,
n_jobs = -1) 
  
# fitting the model for grid search 
grid.fit(xTrain, yTrain) 

Fitting 3 folds for each of 14 candidates, totalling 42 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:   40.9s finished


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7],
                         'n_estimators': [100, 200]},
             verbose=2)

In [100]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [126]:
print_results(grid)

BEST PARAMS: {'max_depth': 7, 'n_estimators': 100}

0.228 (+/-0.046) for {'max_depth': 1, 'n_estimators': 100}
0.224 (+/-0.041) for {'max_depth': 1, 'n_estimators': 200}
0.364 (+/-0.055) for {'max_depth': 2, 'n_estimators': 100}
0.365 (+/-0.06) for {'max_depth': 2, 'n_estimators': 200}
0.448 (+/-0.055) for {'max_depth': 3, 'n_estimators': 100}
0.448 (+/-0.046) for {'max_depth': 3, 'n_estimators': 200}
0.489 (+/-0.038) for {'max_depth': 4, 'n_estimators': 100}
0.498 (+/-0.038) for {'max_depth': 4, 'n_estimators': 200}
0.517 (+/-0.026) for {'max_depth': 5, 'n_estimators': 100}
0.512 (+/-0.035) for {'max_depth': 5, 'n_estimators': 200}
0.529 (+/-0.022) for {'max_depth': 6, 'n_estimators': 100}
0.528 (+/-0.024) for {'max_depth': 6, 'n_estimators': 200}
0.534 (+/-0.027) for {'max_depth': 7, 'n_estimators': 100}
0.534 (+/-0.026) for {'max_depth': 7, 'n_estimators': 200}


### Evaluate your model 

In [127]:
# 'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 100
regressor15 = RandomForestRegressor(n_estimators = 100, random_state = 1,max_depth=7)

In [128]:
regressor15.fit(xTrain, yTrain)

RandomForestRegressor(max_depth=7, random_state=1)

In [129]:
y_pred_final = regressor15.predict(xTest) 

In [130]:
print(metrics.mean_absolute_error(yTest, y_pred_final))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred_final))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred_final)))

25.20909250254835
1095.4680719359803
33.097856002103526


## Task 5

### Use any Feature Selection Technique and observe if the model created by selected features performs better than the previous model

In [138]:
# boruta_selector = BorutaPy(forest, n_estimators='auto', verbose=2)
boruta_selector = BorutaPy(regressor15, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
boruta_selector.fit(np.array(xTrain), np.array(yTrain))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	39
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	15
Tentative: 	10
Rejected: 	14
Iteration: 	9 / 100
Confirmed: 	15
Tentative: 	10
Rejected: 	14
Iteration: 	10 / 100
Confirmed: 	15
Tentative: 	10
Rejected: 	14
Iteration: 	11 / 100
Confirmed: 	15
Tentative: 	10
Rejected: 	14
Iteration: 	12 / 100
Confirmed: 	15
Tentative: 	8
Rejected: 	16
Iteration: 	13 / 100
Confirmed: 	15
Tentative: 	8
Rejected: 	16
Iteration: 	14 / 100
Confirmed: 	15
Tentative: 	8
Rejected: 	16
Iteration: 	15 / 100
Confirmed: 	15
Tentative: 	8
Rejected: 	16
Iteration: 	16 / 100
Confirmed: 	15
Tentative: 	8
Re

BorutaPy(estimator=RandomForestRegressor(max_depth=7, n_estimators=92,
                                         random_state=RandomState(MT19937) at 0x7FA09BC1DEB8),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FA09BC1DEB8, verbose=2)

In [139]:
# check selected features
print("Selected Features: ", boruta_selector.support_)
 
# check ranking of features
print("Ranking: ",boruta_selector.ranking_)

print("No. of significant features: ", boruta_selector.n_features_)

Selected Features:  [False  True  True  True  True  True False False False  True  True  True
 False  True False  True  True  True  True  True False False False  True
 False  True False  True  True False False False False False False False
 False False False]
Ranking:  [ 5  1  1  1  1  1 12  7 11  1  1  1  7  1  4  1  1  1  1  1 10  9  2  1
  5  1  3  1  1  2 15 16 13 14 17 18 20 21 19]
No. of significant features:  18


In [141]:
selected_rfe_features = pd.DataFrame({'Feature':list(xTrain.columns),
                                      'Ranking':boruta_selector.ranking_})
selected_rfe_features.sort_values(by='Ranking')

,Feature,Ranking
19,PctPrivateCoverage,1
25,PctBlack,1
18,PctUnemployed16_Over,1
17,PctEmployed16_Over,1
16,PctBachDeg25_Over,1
15,PctHS25_Over,1
13,PctHS18_24,1
23,PctPublicCoverageAlone,1
10,AvgHouseholdSize,1
11,PercentMarried,1


In [142]:
X_important_train = boruta_selector.transform(np.array(xTrain))
X_important_test = boruta_selector.transform(np.array(xTest))

In [144]:
rf_important = RandomForestRegressor(n_estimators = 100, random_state = 1,max_depth=7)

# Train the new classifier on the new dataset containing the most important features
rf_important.fit(X_important_train, yTrain)


RandomForestRegressor(max_depth=7, random_state=1)

In [146]:
y_pred_final_imp = rf_important.predict(X_important_test) 

In [147]:
print(metrics.mean_absolute_error(yTest, y_pred_final_imp))

#print result of MSE
print(metrics.mean_squared_error(yTest, y_pred_final_imp))

#print result of RMSE
print(np.sqrt(metrics.mean_squared_error(yTest, y_pred_final_imp)))

25.18581061903711
1102.4580491252927
33.20328370997804


That is the end of this assignment. We hope you learnt a lot along the way. 
Make sure you complete the other 2 parts (leaderboard submission and quiz) as well.

Now, download this notebook and submit it on the platform: https://dphi.tech/practice/challenge/52#notebooks 
